In [2]:
import sddk
import pandas as pd
pd.options.display.max_columns = 1000 # to see all columns
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')
import nltk

In [3]:
LIST = gpd.read_parquet("../data/large_data/LIST_classified.parquet")

In [4]:
# records with valid EDH-ID
edh_ids_list = LIST[LIST["EDH-ID"].notnull()]["EDH-ID"].tolist()
len(edh_ids_list)

81266

In [5]:
# records with valid & UNIQUE EDH-ID
edh_ids_unique = LIST[LIST["EDH-ID"].notnull()]["EDH-ID"].unique().tolist()
len(edh_ids_unique)

79899

In [6]:
# EDH-IDs appearing at least twice:
dup_ids = [id_tup[0] for id_tup in nltk.FreqDist(edh_ids_list).most_common() if id_tup[1] > 1]
len(dup_ids)

1136

In [7]:
# dataframe of duplicate rows (i.e. every second instance of the same id
LIST[LIST["EDH-ID"].notnull()].loc[LIST.duplicated(subset="EDH-ID"), :]

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,large_city_pop,medium_city_pop,small_city_pop,urban_context,urban_context_city,urban_context_pop_est,type_of_inscription_auto,type_of_inscription_auto_prob
449357,LIST-449358,EDCS-28500267,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]T[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,T,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0,None,0.0
449358,LIST-449359,EDCS-28500268,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]SA[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,SA,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0,None,0.0
449359,LIST-449360,EDCS-28500269,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]M[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,M,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,medium,Delos,18364.0,None,0.0
449360,LIST-449361,EDCS-28500271,HD064154,https://www.trismegistos.org/text/178167,599588,[---]S[---] // [---]T[---] // [---]SA[---] // ...,]N[,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,S T SA M G N I DE O K OR CVF,N,[ ]S[ ] / / [ ]T[ ] / / [ ]SA[ ] / / [ ]M[ ] /...,Achaia,Delos / Ortygia,None,None,None,None,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-18,17 cm,None,no image,NaN,None,\n S T SA M G N I DE O K OR CVF \n,None,NaN,1000.0,143.0,[ ],NULL,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Delos,False,Nótio Aigaío,False,Dhílos,False,NULL,False,L,None,None,NaN,NaN,25.2661,37.4036,False,POINT (25.26610 37.40360),False,58112.0,18364.0,2290.0,m

In [8]:
# remove the duplicates
# (using pandas deduplicated() method)
LIST = LIST.loc[~(LIST.duplicated(subset="EDH-ID")) | (LIST["EDH-ID"].isnull())]
len(LIST)

528484

In [9]:
EDH_overlap_all = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].notnull())]
len(EDH_overlap_all)

76583

In [10]:
len(LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].isnull())])

3316

In [11]:
len(LIST[(LIST["EDH-ID"].isnull()) & (LIST["EDCS-ID"].notnull())])

448585

# Save and export

In [ ]:
# locally as parquet
LIST.to_parquet("../data/large_data/LIST_v0.2.parquet")

In [ ]:
# login to sciencedata.dk
s = sddk.cloudSession("sciencedata.dk", "SDAM_root/SDAM_data/LIRE", "648597@au.dk")

In [ ]:
# upload data to sciencedata.dk
s.write_file("LIST_v0-2.parquet", LIST)